In [ ]:
!pip install openpyxl
!pip install pandas


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpy

In [12]:
import pandas as pd
import re

In [19]:
def load_mappings(excel_url: str, sheet_name: str = "CRM_S4_TABLE_FIELDS_MAPPING"):
    df = pd.read_excel(excel_url, sheet_name=sheet_name)
    rules = df.dropna(subset=["CRM_TABNAME", "S4_TABNAME"]) \
              .set_index("CRM_TABNAME")["S4_TABNAME"].to_dict()
    field_mapping = df.dropna(subset=["CRM_FIELDNAME", "S4_FIELDNAME"]) \
                      .set_index("CRM_FIELDNAME")["S4_FIELDNAME"].to_dict()
    return rules, field_mapping

In [20]:
def transform_query(source_query, rules, field_mapping):
    query = re.sub(r'\s+', ' ', source_query.strip())

    # Detect SINGLE
    single = bool(re.search(r'\bSINGLE\b', query, re.IGNORECASE))

    # Detect FOR ALL ENTRIES
    fae_match = re.search(r'FOR ALL ENTRIES IN (\w+)', query, re.IGNORECASE)
    fae_clause = f'FOR ALL ENTRIES IN {fae_match.group(1).upper()}' if fae_match else ''

    # Detect UP TO ... ROWS
    up_to_match = re.search(r'UP TO (\w+) ROWS', query, re.IGNORECASE)
    up_to_clause = f'UP TO {up_to_match.group(1)} ROWS' if up_to_match else ''

    # Extract fields
    fields = []
    select_fields_match = re.search(r'SELECT (.*?) FROM', query, re.IGNORECASE)
    if select_fields_match:
        fields_str = select_fields_match.group(1).strip()
        fields = [f.strip() for f in (fields_str.split(',') if ',' in fields_str else fields_str.split()) if f.strip()]
    else:
        fields_match = re.search(r'FIELDS (.*?)(WHERE|INTO|$)', query, re.IGNORECASE)
        if fields_match:
            fields_str = fields_match.group(1).strip().rstrip(',')
            fields = [f.strip() for f in (fields_str.split(',') if ',' in fields_str else fields_str.split()) if f.strip()]

    fields = [f for f in fields if f.strip().upper() != 'SINGLE']

    # Transform fields
    transformed_fields = []
    for field in fields:
        field_for_map = re.sub(r'@DATA\((.*?)\)', r'\1', field).lstrip('@')
        if '~' in field_for_map:
            _, fname = field_for_map.split('~', 1)
        else:
            fname = field_for_map
        fname = fname.strip().upper()
        mapped = field_mapping.get(fname, fname)   # ✅ Preserve original if not mapped
        transformed_fields.append(f'{mapped} AS {fname}')

    # INTO clause
    into_clause = ''
    into_match = re.search(r'INTO\s+(TABLE\s+)?((@?DATA\([^)]+\))|(@?\w+))', query, re.IGNORECASE)
    if into_match:
        into_var = into_match.group(2)
        into_clause = f'INTO TABLE {into_var}' if into_match.group(1) else f'INTO {into_var}'

    # WHERE clause
    where_clause = ''
    where_match = re.search(r'WHERE (.*?)(FOR ALL ENTRIES|INTO|UP TO|$)', query, re.IGNORECASE)
    if where_match:
        where_cond = where_match.group(1).strip()

        def replace_tilde(m):
            return field_mapping.get(m.group(2).upper(), m.group(2).upper())

        where_cond = re.sub(r'(\w+)~(\w+)', replace_tilde, where_cond)
        where_clause = f'WHERE {where_cond}'

    # FROM + JOINs
    from_clause = query.split("FROM", 1)[1]  # everything after FROM
    # Replace table names
    for crm_table, s4_table in rules.items():
        from_clause = re.sub(rf'\b{crm_table}\b', s4_table, from_clause, flags=re.IGNORECASE)

    # Final query
    parts = []
    parts.append('SELECT SINGLE' if single else 'SELECT')
    parts.append(', '.join(transformed_fields))
    parts.append('FROM ' + from_clause.strip())
    if into_clause:
        parts.append(into_clause)
    if fae_clause:
        parts.append(fae_clause)
    if up_to_clause:
        parts.append(up_to_clause)
    if where_clause:
        parts.append(where_clause)

    return ' '.join(parts)


In [25]:
if __name__ == "__main__":
    excel_url = "https://raw.githubusercontent.com/srinivashj1/code/main/Select%20Queries%201.xlsx"
    
    # Make sure the sheet name matches exactly the tab name in your Excel file
    rules, field_mapping = load_mappings(excel_url, sheet_name="CRM_S4_TABLE_FIELDS_MAPPING")
    
    test_queries = [
        '''SELECT A~GUID C~SALES_ORG, KNUMV FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET 
           INTO TABLE LT_ORG WHERE A~GUID EQ LV_GUID''',

        '''SELECT A~GUID, C~SALES_ORG FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET 
           INTO TABLE @DATA(LT_ORG) WHERE A~GUID EQ @LV_GUID''',

        '''SELECT A~GUID, C~SALES_ORG FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET 
           INTO TABLE @LT_ORG WHERE A~GUID EQ @LV_GUID''',

        '''SELECT FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B ON B~GUID_HI  = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID  = B~GUID_SET 
           FIELDS A~GUID,  C~SALES_ORG 
           WHERE  A~GUID EQ  @LV_GUID INTO TABLE @LT_ORG''',

        '''SELECT A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET 
           FOR ALL ENTRIES IN LT_GUID INTO TABLE LT_ORG 
           WHERE A~GUID EQ LT_GUID-GUID''',

        '''SELECT SINGLE A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET 
           INTO LS_ORG WHERE A~GUID EQ LV_GUID''',

        '''SELECT SINGLE A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID = B~GUID_SET 
           INTO TABLE LT_ORG UP TO XXXX ROWS WHERE A~GUID EQ LV_GUID''',

        '''SELECT A~GUID C~SALES_ORG FROM CRMD_ORDERADM_H AS A 
           INNER JOIN CRMD_LINK AS B  ON B~GUID_HI  = A~GUID 
           INNER JOIN CRMD_ORGMAN AS C ON C~GUID  = B~GUID_SET 
           INTO  LS_ORG WHERE  A~GUID EQ   LV_GUID.'''
    ]

    for i, q in enumerate(test_queries, 1):
        print(f'R{i}:', transform_query(q, rules, field_mapping))


R1: SELECT GUID C~SALES_ORG AS GUID C~SALES_ORG, PRICING_DOCUMENT AS KNUMV FROM CRMS4D_SERV_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMS4D_SERV_I AS C ON C~GUID = B~GUID_SET INTO TABLE LT_ORG WHERE A~GUID EQ LV_GUID INTO TABLE LT_ORG WHERE ITEM_GUID_CHAR EQ LV_GUID
R2: SELECT ITEM_GUID_CHAR AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMS4D_SERV_I AS C ON C~GUID = B~GUID_SET INTO TABLE @DATA(LT_ORG) WHERE A~GUID EQ @LV_GUID INTO TABLE @DATA(LT_ORG) WHERE ITEM_GUID_CHAR EQ @LV_GUID
R3: SELECT ITEM_GUID_CHAR AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JOIN CRMS4D_SERV_I AS C ON C~GUID = B~GUID_SET INTO TABLE @LT_ORG WHERE A~GUID EQ @LV_GUID INTO TABLE @LT_ORG WHERE ITEM_GUID_CHAR EQ @LV_GUID
R4: SELECT ITEM_GUID_CHAR AS GUID, SALES_ORG AS SALES_ORG FROM CRMS4D_SERV_H AS A INNER JOIN CRMD_LINK AS B ON B~GUID_HI = A~GUID INNER JO